# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
# Import API key
from api_keys import g_key

import numpy as np
import requests
import gmaps
import os
from pprint import pprint



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Download CSV file
cities_file="../output_data/cities.csv"
cities_df = pd.read_csv(cities_file)
cities_df

Unnamed: 0  City Name  Latitude  Longitude  Maximum Temperature  \
0             0      chara     56.91     118.26                57.49   
1             1       ayan     56.45     138.17                52.38   
2             2     tiznit     29.58      -9.50                71.10   
3             3  hithadhoo     -0.60      73.08                84.02   
4             4  loandjili     -4.76      11.86                71.33   
..          ...        ...       ...        ...                  ...   
566         611     wanaka    -44.70     169.15                44.74   
567         612   carthage     36.86      10.33                69.80   
568         613      vardo     70.37      31.11                57.20   
569         614     losevo     50.68      40.05                62.06   
570         615    merrill     45.18     -89.68                66.20   

     Humidity  Cloudiness  Wind Speed Country        Date  
0          66         100        5.19      RU  1595299548  
1          94         100       15.39      RU  1595299548  
2          52           1        1.77      MA  1595299549  
3          77         100       11.52      MV  1595299549  
4          89          39        2.55      CG  1595299549  
..        ...         ...         ...     ...         ...  
566        81          89        5.86      NZ  1595299612  
567        64           0        4.50      TN  1595299612  
568       100          75        6.93      NO  1595299612  
569        80           0        3.87      RU  1595299612  
570        87          40        3.58      US  1595299612  

[571 rows x 10 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Create heatmap variables
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)

In [4]:
#Create Heatmap
humidity_map= gmaps.figure()
heat = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
humidity_map.add_layer(heat)
humidity_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Filter out less than ideal conditions
perfect = cities_df[cities_df['Maximum Temperature'] > 60]
perfect = perfect[perfect['Maximum Temperature'] < 80]
perfect = perfect[perfect['Cloudiness'] < 50]
perfect = perfect[perfect['Cloudiness'] > 25]
perfect = perfect[perfect['Wind Speed'] < 10]
perfect = perfect[perfect['Humidity'] < 70]
perfect


Unnamed: 0     City Name  Latitude  Longitude  Maximum Temperature  \
85           96        polson     47.69    -114.16                78.01   
157         172      almaznyy     48.04      40.05                68.00   
239         264        gurupi    -11.73     -49.07                69.82   
407         440  ust-barguzin     53.45     108.98                60.04   
447         483          aksu     41.12      80.26                76.68   

     Humidity  Cloudiness  Wind Speed Country        Date  
85         51          46        2.17      US  1595299389  
157        64          28        8.95      RU  1595299567  
239        46          39        4.05      BR  1595299578  
407        53          35        3.04      RU  1595299596  
447        41          26        3.80      CN  1595299600

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Variables to for Google maps search
target_type = "hotel"
city_coords = perfect["Latitude"],perfect["Longitude"]
radius = 50000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create lists
hotel_city = perfect["City Name"].tolist()
hotel_country = perfect["Country"].tolist()
lat_list = perfect["Latitude"].tolist()
lng_list = perfect["Longitude"].tolist()
hotel_name = []
hotel_lat = []
hotel_lng = []

#Pull up data from each city
for x in range(0,len(lat_list)):
    params = {
        "location": f'{lat_list[x]},{lng_list[x]}',
        "types": target_type,
        "radius": radius,
        "key": g_key
    }

    response = requests.get(base_url, params)
    nearby_hotels = response.json()
    
#Append them to lists
    hotel_name.append(nearby_hotels["results"][0]['name'])
    hotel_lat.append(nearby_hotels["results"][0]["geometry"]["location"]["lat"])
    hotel_lng.append(nearby_hotels["results"][0]["geometry"]["location"]["lng"])



In [7]:
#Create DataFrame
Hoteldata = {'Hotel Name': hotel_name,'City': hotel_city,'Country': hotel_country,
        'Lat': hotel_lat,
        'Lng': hotel_lng}


hotel_df = pd.DataFrame(Hoteldata, columns = ['Hotel Name', 'City', 'Country', 'Lat', 'Lng'])
hotel_df

Hotel Name          City Country        Lat         Lng
0           Polson        polson      US  47.693200 -114.163128
1          Shakhty      almaznyy      RU  47.708607   40.216038
2           Gurupi        gurupi      BR -11.728375  -49.068608
3     Ust-Barguzin  ust-barguzin      RU  53.412359  109.033908
4  Aksu Prefecture          aksu      CN  41.168779   80.260605

In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
humidity_map.add_layer(markers)
# Display figure
humidity_map

#Click on marker to get info

Figure(layout=FigureLayout(height='420px'))